In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tt
import experiments_stuff
from bilinear_package.src import primitives
from bilinear_package.src.hadamard_product import generalizedApproximateHadamardProduct
from bilinear_package.src.convolution import approximateConvolution, preciseConvolution, countFourier, countInverseFourier, approximateCycleConvolution
import time

In [3]:
from newton_experiment_primitives import create_exponential_grid, create_newtonial_potential_grid

In [4]:
np.logspace(2, 6, 5, base = 2, dtype = int)

array([ 4,  8, 16, 32, 64])

In [5]:
exp_grid = create_exponential_grid(3, 4096, 40)

In [6]:
inv_grid = create_newtonial_potential_grid(3, 4096, 40)

In [7]:
tt.vector.from_list(exp_grid)

This is a 3-dimensional tensor 
r(0)=1, n(0)=4096 
r(1)=20, n(1)=4096 
r(2)=20, n(2)=4096 
r(3)=1 

In [8]:
tt.vector.from_list(inv_grid)

This is a 3-dimensional tensor 
r(0)=1, n(0)=4096 
r(1)=15, n(1)=4096 
r(2)=14, n(2)=4096 
r(3)=1 

In [42]:
A = 40

answer = []

for grid_side in np.logspace(8, 10, 3, base = 2, dtype = int):
    h = (2 * A) / grid_side
    tensor1 = create_exponential_grid(3, grid_side + 2, A + h / 2)
    tensor2 = create_newtonial_potential_grid(3, 2 * grid_side + 2, h * grid_side + h / 2)
    tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(3)])
    bench = time.time()
    approx = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(2, dtype=int) * 15, seed=255)
    print(time.time() - bench)
    approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(3)])
    answer.append(approx)

2.5382471084594727
6.061351299285889
11.025248289108276
28.29969048500061
53.79193902015686
120.85264420509338


In [43]:
from newton_experiment_primitives import compressTTTensor

kek = []

for i in range(1, len(answer)):
    A = answer[i - 1].copy()
    B = answer[i].copy()
    A = tt.vector.from_list(A)
    B_prime = tt.vector.from_list(compressTTTensor(B))
    kek.append(tt.vector.norm(A - B_prime))


In [46]:
kek = np.array(kek)

In [54]:
print(np.log((kek[1:] / kek[:-1])) / np.log(2))

[4.49503219 4.49751372 4.49875609 4.49937849]


In [58]:
A = 40

answer = []

for grid_side in np.logspace(8, 13, 6, base = 2, dtype = int):
    h = (2 * A) / grid_side
    tensor1 = create_exponential_grid(4, grid_side + 2, A + h / 2)
    tensor2 = create_newtonial_potential_grid(4, 2 * grid_side + 2, h * grid_side + h / 2)
    tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(4)])
    bench = time.time()
    approx = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(3, dtype=int) * 15, seed= 271 + grid_side)
    print(time.time() - bench)
    approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(4)])
    answer.append(approx)

2.0796468257904053
5.132342576980591
9.101766586303711
20.543912410736084
42.34511852264404
88.25273656845093


/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, i, :] = kernel[:, 2 * i, :]


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [65]:
kek = np.empty(0)


for i in range(2, len(answer)):
    A = answer[i - 2].copy()
    B = answer[i - 1].copy()
    C = answer[i].copy()
    A = tt.vector.from_list(A)
    B = tt.vector.from_list(compressTTTensor(B))
    C = tt.vector.from_list(compressTTTensor(compressTTTensor(C)))
    print(tt.vector.norm(B - C) / tt.vector.norm(A - B))

/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, i, :] = kernel[:, 2 * i, :]


15.963145818552455
15.981546579578419
15.990790577233247
15.99501390115569


In [6]:
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor


D = 40

for d in range(1, 6):
    answer = []
    for grid_side in np.logspace(6, 12, 7, base = 2, dtype = int):
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        bench = time.time()
        approx = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 20, seed= 271 * d + grid_side)
        approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(d)])
        print(time.time() - bench)
        for i in range(len(approx)):
            approx[i] = approx[i] * h
        answer.append(approx)
        print()
    print()
    for mid in range(1, 5):
        A = answer[mid - 1].copy()
        B = answer[mid].copy()
        C = answer[mid + 1].copy()
        A = tt.vector.from_list(interpolateTTTensor(A))
        B = tt.vector.from_list(B)
        C = tt.vector.from_list(compressTTTensor(C))
        print(tt.vector.norm(A))
        print(tt.vector.norm(B))
        print(tt.vector.norm(C))
        print(tt.vector.norm(B - C) / tt.vector.norm(A - B))
        print()

This is a 1-dimensional tensor 
r(0)=1, n(0)=130 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=1 

0.000431060791015625

This is a 1-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=1 

0.0004978179931640625

This is a 1-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=1 

0.0003705024719238281

This is a 1-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=1 

0.0005507469177246094

This is a 1-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=4098 
r(1)=1 

0.0033006668090820312

This is a 1-dimensional tensor 
r(0)=1, n(0)=4098 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=8194 
r(1)=1 

0.005307197570800781


1943.6645629493287
1957.6886609304872
1958.757588591865
0.10101037408637417

2767.4208531736153
2768.9439570825966
2767.584347403369
0.7447442790501

/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:43: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, 2 * i, :] = kernel[:, i, :]
/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, 2 * i - 1,
/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:61: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, i, :] = kernel[:, 2 * i, :]


This is a 2-dimensional tensor 
r(0)=1, n(0)=130 
r(1)=13, n(1)=130 
r(2)=1 

This is a 2-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=17, n(1)=258 
r(2)=1 

0.04242062568664551

This is a 2-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=14, n(1)=258 
r(2)=1 

This is a 2-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=18, n(1)=514 
r(2)=1 

0.16410422325134277

This is a 2-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=16, n(1)=514 
r(2)=1 

This is a 2-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=19, n(1)=1026 
r(2)=1 

0.3608131408691406

This is a 2-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=17, n(1)=1026 
r(2)=1 

This is a 2-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=20, n(1)=2050 
r(2)=1 

0.6645677089691162

This is a 2-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=18, n(1)=2050 
r(2)=1 

This is a 2-dimensional tensor 
r(0)=1, n(0)=4098 
r(1)=20, n(1)=4098 
r(2)=1 

1.182795524597168

This is a 2-dimensional tensor 
r(0)=1, n(0)=4098 
r(1)=19, n(1)=4098 
r(2)=1 

This is a 2-dimensional tenso

: 

: 

In [ ]:
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor


D = 40

for d in range(1, 6):
    answer = []
    for grid_side in np.logspace(7, 12, 6, base = 2, dtype = int):
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        bench = time.time()
        approx = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 20, seed= 271 * d + grid_side)
        approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(d)])
        print(time.time() - bench)
        for i in range(len(approx)):
            approx[i] = approx[i] * h
        answer.append(approx)
        print()
    print()
    for mid in range(1, 5):
        A = answer[mid - 1].copy()
        B = answer[mid].copy()
        C = answer[mid + 1].copy()
        A = tt.vector.from_list(interpolateTTTensor(A))
        B = tt.vector.from_list(B)
        C = tt.vector.from_list(compressTTTensor(C))
        print(tt.vector.norm(A))
        print(tt.vector.norm(B))
        print(tt.vector.norm(C))
        print(tt.vector.norm(B - C) / tt.vector.norm(A - B))
        print()

In [ ]:
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor


D = 40

for d in range(1, 6):
    answer = []
    for grid_side in np.logspace(7, 12, 6, base = 2, dtype = int):
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        bench = time.time()
        approx = approximateCycleConvolution(tensor1, tensor2, desired_ranks=np.ones(d - 1, dtype=int) * 20, seed= 271 * d + grid_side)
        approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(d)])
        print(time.time() - bench)
        for i in range(len(approx)):
            approx[i] = approx[i] * h
        answer.append(approx)
        print()
    print()
    for mid in range(1, 5):
        A = answer[mid - 1].copy()
        B = answer[mid].copy()
        C = answer[mid + 1].copy()
        A = tt.vector.from_list(interpolateTTTensor(A))
        B = tt.vector.from_list(B)
        C = tt.vector.from_list(compressTTTensor(C))
        print(tt.vector.norm(A))
        print(tt.vector.norm(B))
        print(tt.vector.norm(C))
        print(tt.vector.norm(B - C) / tt.vector.norm(A - B))
        print()

In [7]:
from newton_experiment_primitives import interpolateTTTensor, compressTTTensor


D = 40

for d in range(1, 6):
    answer = []
    for grid_side in np.logspace(7, 11, 5, base = 2, dtype = int):
        h = (2 * D) / grid_side
        tensor1 = create_exponential_grid(d, grid_side + 2, D + h / 2)
        tensor2 = create_newtonial_potential_grid(d, 2 * grid_side + 2, h * grid_side + h / 2)
        print(tt.vector.from_list(tensor1))
        print(tt.vector.from_list(tensor2))
        tensor1 = primitives.twoSidedPaddingTTTensor(tensor1, [(0, grid_side) for _ in range(d)])
        bench = time.time()
        tensor1 = countFourier(tensor1)
        tensor2 = countFourier(tensor2)
        approx = tt.vector.to_list(tt.multifuncrs2([tt.vector.from_list(tensor1), tt.vector.from_list(tensor2)], lambda x : x[:, 0] * x[:, 1], eps=1e-6, verb = 0))
        approx = countInverseFourier(approx)
        approx = primitives.twoSidedCuttingTTTensor(approx, [(0, grid_side + 1) for _ in range(d)])
        print(time.time() - bench)
        for i in range(len(approx)):
            approx[i] = approx[i] * h
        answer.append(approx)
        print()
    print()
    for mid in range(1, 4):
        A = answer[mid - 1].copy()
        B = answer[mid].copy()
        C = answer[mid + 1].copy()
        A = tt.vector.from_list(interpolateTTTensor(A))
        B = tt.vector.from_list(B)
        C = tt.vector.from_list(compressTTTensor(C))
        print(tt.vector.norm(A))
        print(tt.vector.norm(B))
        print(tt.vector.norm(C))
        print(tt.vector.norm(B - C) / tt.vector.norm(A - B))
        print()

This is a 1-dimensional tensor 
r(0)=1, n(0)=130 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=1 

0.005860090255737305

This is a 1-dimensional tensor 
r(0)=1, n(0)=258 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=1 

0.006443023681640625

This is a 1-dimensional tensor 
r(0)=1, n(0)=514 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=1 

0.008672952651977539

This is a 1-dimensional tensor 
r(0)=1, n(0)=1026 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=1 

0.003570079803466797

This is a 1-dimensional tensor 
r(0)=1, n(0)=2050 
r(1)=1 

This is a 1-dimensional tensor 
r(0)=1, n(0)=4098 
r(1)=1 

0.006943941116333008


1943.6645629493285
1957.6886609304875
1958.7575885918652
0.1010103740863774

2767.4208531736153
2768.943957082597
2767.5843474033686
0.7447442790504863

3915.054568990724
3913.1364658742204
3911.4274085351562
0.8286365472503908

This is a 2-dimensional tensor 
r(0)=1, n(0)=130 
r(1)=13, n(1)=1

In [50]:
from newton_experiment_primitives import interpolateTTTensor

lol = np.empty(0)

for i in range(1, len(answer)):
    A = answer[i - 1].copy()
    B = answer[i].copy()
    A = tt.vector.from_list(interpolateTTTensor(A))
    B_prime = tt.vector.from_list(B)
    lol = np.append(lol, tt.vector.norm(A - B_prime))


/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, 2 * i, :] = kernel[:, i, :]
/home/divanik/Desktop/CourseWork/Randomized_tensor_bilinear_operations/newton_experiment_primitives.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  answer[:, 2 * i - 1,


In [51]:
print((lol[1:] / lol[:-1]) / 8)

[2.82203552 2.82522865 2.82682689 2.827628  ]


In [9]:
A = 40

for number in np.logspace(8, 13, 6, base = 2, dtype = int):
    tensor1 = create_exponential_grid(3, number, A)
    tensor2 = create_newtonial_potential_grid(3, number, A)
    precise = tt.vector.from_list(preciseConvolution(tensor1, tensor2))
    print(precise)
    bench = time.time()
    approx = precise.round(0, 30)
    print(time.time() - bench)

This is a 3-dimensional tensor 
r(0)=1, n(0)=511 
r(1)=210, n(1)=511 
r(2)=210, n(2)=511 
r(3)=1 

6.750958442687988
This is a 3-dimensional tensor 
r(0)=1, n(0)=1023 
r(1)=255, n(1)=1023 
r(2)=238, n(2)=1023 
r(3)=1 

21.29015612602234


: 

: 

In [17]:
A = 40

for number in np.logspace(8, 12, 5, base = 2, dtype = int):
    tensor1 = tt.vector.from_list(countFourier(create_exponential_grid(3, number, A)))
    tensor2 = tt.vector.from_list(countFourier(create_newtonial_potential_grid(3, number, A)))
    bench = time.time()
    approx = tt.multifuncrs2([tensor1, tensor2], lambda x: x[:, 0] * x[:, 1], verb=0)
    print(approx)
    print(time.time() - bench)

This is a 3-dimensional tensor 
r(0)=1, n(0)=256 
r(1)=13, n(1)=256 
r(2)=13, n(2)=256 
r(3)=1 

1.042820692062378
This is a 3-dimensional tensor 
r(0)=1, n(0)=512 
r(1)=13, n(1)=512 
r(2)=13, n(2)=512 
r(3)=1 

2.3004324436187744
This is a 3-dimensional tensor 
r(0)=1, n(0)=1024 
r(1)=13, n(1)=1024 
r(2)=13, n(2)=1024 
r(3)=1 

5.254962682723999


KeyboardInterrupt: 

In [248]:
print(tt.vector.from_list(tensor))

This is a 2-dimensional tensor 
r(0)=1, n(0)=6 
r(1)=6, n(1)=6 
r(2)=1 



In [191]:
def create_2d_exponential_grid(d, A, alpha=1):
    x = np.linspace(-A, A, d, endpoint=True)
    kek = x[:, None] * x[:, None] + x[None, :] * x[None, :] 
    return np.exp(-alpha * np.sqrt(kek))

In [192]:
def create_2d_invertable_grid(d, A, alpha=1):
    x = np.linspace(-A, A, d, endpoint=True)
    kek = x[:, None] * x[:, None] + x[None, :] * x[None, :] 
    return 1 / np.sqrt(kek)

In [239]:
a = create_exponential_grid(6, 30, 116, alpha=0.1)
tt.vector.from_list(a)

This is a 6-dimensional tensor 
r(0)=1, n(0)=30 
r(1)=14, n(1)=30 
r(2)=14, n(2)=30 
r(3)=14, n(3)=30 
r(4)=14, n(4)=30 
r(5)=14, n(5)=30 
r(6)=1 

In [242]:
b = create_invertable_grid(6, 30, 116)
tt.vector.from_list(b)

This is a 6-dimensional tensor 
r(0)=1, n(0)=30 
r(1)=14, n(1)=30 
r(2)=15, n(2)=30 
r(3)=15, n(3)=30 
r(4)=15, n(4)=30 
r(5)=14, n(5)=30 
r(6)=1 

In [244]:
bench = time.time()
for i in range(20):
    approx = approximateConvolution(a, b, desired_ranks=np.ones(5, dtype=int) * 6, seed=255)
print(time.time() - bench)

3.0013227462768555


In [245]:
precise = preciseConvolution(a, b)

In [246]:
bench = time.time()
for i in range(20):
    tt.vector.from_list(precise).round(0, 6)
print(time.time() - bench)

/home/divanik/.local/lib/python3.8/site-packages/tt/core/vector.py:412: ComplexWarning: Casting complex values to real discards the imaginary part
  _tt_f90.tt_f90.dtt_compr2(c.n, c.r, c.ps, self.core, eps, rmax)


17.25714111328125


In [247]:
primitives.ttTensorsUnsymmetricalRelativeComparance(approx, precise)

(0.00033241089159884775+0j)